<a href="https://colab.research.google.com/github/leticiacaixeta/gsi073/blob/main/GSI073_aula0_regressao_logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSI073 - Tópicos Especiais de Inteligência Artificial

## Definição dos dados

In [ ]:
import torch
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np # Importado para uso com a avaliação no numpy

# 1. Carregar dados
iris = sklearn.datasets.load_iris()
X_np = iris.data        # 4 features: sépalas e pétalas
# O reshape é necessário para garantir que y_np seja uma matriz de 2D (N, 1)
y_np = (iris.target == 1).astype(float).reshape(-1, 1)  # 1 se Versicolor, 0 caso contrário

# 1.1 Separar em treino e teste (80% treino, 20% teste)
X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(
    X_np, y_np, test_size=0.2, random_state=42, stratify=y_np
)

# 2. Preparar dados para pytorch
# Converter os conjuntos separados para tensores do PyTorch
X_train = torch.tensor(X_train_np, dtype=torch.float32)
y_train = torch.tensor(y_train_np, dtype=torch.float32)

X_test = torch.tensor(X_test_np, dtype=torch.float32)
y_test = torch.tensor(y_test_np, dtype=torch.float32)

print(f"Dimensões de Treino (X_train): {X_train.shape}, (y_train): {y_train.shape}")
print(f"Dimensões de Teste (X_test): {X_test.shape}, (y_test): {y_test.shape}")

## Definição do modelo e treinamento

In [ ]:
# 3. Definir modelo: regressão logística
modelo = torch.nn.Linear(4, 1)  # 4 features → 1 saída (probabilidade de ser Versicolor)

# 4. Definir função de perda e algoritmo de otimização
funcao_perda = torch.nn.BCEWithLogitsLoss()  # Combinação de sigmoid + BCE
optimizer = torch.optim.SGD(modelo.parameters(), lr=0.1)

## Execução do treinamento

In [ ]:
# 5. Treino
print("Iniciando Treinamento...")
for epoch in range(1000):
    optimizer.zero_grad() # reseta gradiente senão acumula

    # Forward pass usando APENAS dados de treino
    outputs = modelo(X_train)
    loss = funcao_perda(outputs, y_train)

    # Backward pass e otimização
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/1000], Loss: {loss.item():.4f}')

print("Treinamento concluído.")

In [ ]:
# PRÁTICA 5: Adicionar uma célula de código para avaliar se o modelo aprendido está bom
print("\n--- Avaliação do Modelo no Conjunto de Teste ---")

# Coloca o modelo em modo de avaliação
modelo.eval()

# Desativa o cálculo de gradientes (economiza recursos na avaliação)
with torch.no_grad():

    # 1. Fazer predições (logits) no conjunto de TESTE
    outputs_test = modelo(X_test)

    # 2. Aplicar Sigmoid para obter probabilidades (entre 0 e 1)
    probabilities = torch.sigmoid(outputs_test)

    # 3. Converter para classes binárias (limite de 0.5)
    predictions = (probabilities >= 0.5).float()

    # 4. Calcular Acurácia (Accuracy)
    correct_predictions = (predictions == y_test).sum().item()
    total_predictions = y_test.size(0)
    accuracy = correct_predictions / total_predictions

    # 5. Calcular a Perda (Loss) no teste
    loss_test = funcao_perda(outputs_test, y_test).item()

    print(f'Loss no Conjunto de Teste: {loss_test:.4f}')
    print(f'Acurácia do Modelo: {accuracy*100:.2f}% ({correct_predictions} de {total_predictions} corretos)')

In [ ]:
# PRÁTICA 6: Descubra o que tem dentro de nn.Linear
print("\n--- Conteúdo de torch.nn.Linear ---")

# A camada torch.nn.Linear implementa a operação fundamental da regressão linear:
# Saída = (Entrada * Pesos) + Viés/Bias
# Ela contém dois parâmetros (tensores) aprendidos: weight e bias.

# 1. Pesos (Weight): A matriz de coeficientes do modelo
print("1. Pesos (Weight) - Coeficientes (W):")
print(modelo.weight.data)
print(f"Shape: {modelo.weight.data.shape}")

# 2. Bias (Viés): O termo de intercepto (b)
print("\n2. Bias (b) - Intercepto:")
print(modelo.bias.data)
print(f"Shape: {modelo.bias.data.shape}")

# O modelo de Regressão Logística é: P(Y=1) = Sigmoid(X * W^T + b)
# Os valores acima (W e b) foram otimizados pelo algoritmo SGD.